In [1]:
%load_ext autoreload
%autoreload 2    

# Setup

In [6]:
DB_NAME = "./memories.db"

In [7]:
from broai.prompt_management.core import PromptGenerator
from broai.prompt_management.interface import Persona, Instructions, Examples, Example
from pydantic import BaseModel, Field
from typing import List
from broai.experiments.bro_agent import BroAgent
import json
from broai.interface import Context, Contexts
from broai.experiments.vector_store import DuckVectorStore

In [8]:
from package.jargon_store import JargonStore, JargonRecord

In [9]:
from broai.experiments.cross_encoder import ReRanker
rr = ReRanker()

/tmp/ipykernel_14426/2186289470.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: ReRanker
  rr = ReRanker()


In [10]:
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension
baai_em = BAAIEmbedding()

/tmp/ipykernel_14426/2466456318.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 226719.14it/s]


In [25]:
raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)
jargon_memory = JargonStore(db_name=DB_NAME, table="jargon_memory")

/tmp/ipykernel_14426/1440006260.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  raw_memory = DuckVectorStore(db_name=DB_NAME, table="raw_memory", embedding=baai_em)
/tmp/ipykernel_14426/1440006260.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  enrich_memory = DuckVectorStore(db_name=DB_NAME, table="enrich_memory", embedding=baai_em)
/tmp/ipykernel_14426/1440006260.py:3: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  longterm_memory = DuckVectorStore(db_name=DB_NAME, table="longterm_memory", embedding=baai_em)


# Agent Flows
- ContextEnricher
- JargonExtractor

In [42]:
from agents.context_enricher import ContextEnricher, InputContext
from agents.jargon_extractor import JargonExtractor, InputContext

In [46]:
ContextEnricher.run(InputContext(context="This is a context for BroAI."))

Summary(summary='Not enough context provided')

In [28]:
JargonExtractor.run(InputContext(context="This is a context for BroAI."))

Jargons(jargons=[Jargon(jargon='BroAI', evidence='This is a context for BroAI', explanation='BroAI is likely an artificial intelligence system or model')])

In [3]:
# import os
# os.remove(DB_NAME)

# Parsing markdown

In [18]:
from broai.experiments.chunk import split_markdown, consolidate_markdown, get_markdown_sections, split_overlap
from broai.interface import Context, Contexts

In [19]:
with open("./docs/storm.md", "r") as f:
    markdown_text = f.read()

In [20]:
chunks = split_markdown(markdown_text)

Markdown headings: max(4)


/tmp/ipykernel_14426/3854812085.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(markdown_text)


In [21]:
consolidated_chunks = consolidate_markdown(chunks)

/tmp/ipykernel_14426/1728333434.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)


In [22]:
sections = get_markdown_sections(consolidated_chunks)

/tmp/ipykernel_14426/4124938538.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)


In [23]:
contexts = Contexts()
source = ".docs/test1/storm.md"
for section, chunk in zip(sections, consolidated_chunks):
    contexts.add_context(Context(context=chunk, metadata={"section": section, "source": source, "type": "document"}))
len(contexts.contexts)

50

In [24]:
new_contexts = split_overlap(contexts.contexts, max_tokens=1000, overlap=300)
new_contexts = Contexts(contexts=new_contexts)
len(new_contexts.contexts)

/tmp/ipykernel_14426/2598309194.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts.contexts, max_tokens=1000, overlap=300)


63

# Create: raw_memory

In [30]:
raw_memory.add_contexts(new_contexts.contexts)

In [32]:
raw_memory.read_all().head(3)

,id,context,metadata,type,embedding,created_at,updated_at
0,c0563185-3f31-4070-b5fc-02a640ebbf9c,# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024\n\nAss...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.04901123, -0.010894775, -0.02658081, -0.02...",2025-04-27 23:49:57.655311,2025-04-27 23:49:57.655311
1,f7bb69b1-eacb-4727-bbb4-88eb928a812c,Abstract\n\nWe study how to apply large langua...,"{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[0.0079193115, -0.016906738, -0.025924683, 0.0...",2025-04-27 23:49:57.655360,2025-04-27 23:49:57.655360
2,13092ee5-9276-4811-bab9-47cab9aed21d,1 Introduction\n\nLarge language models (LLMs)...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.055419922, 0.009933472, -0.023742676, -0.0...",2025-04-27 23:49:57.655377,2025-04-27 23:49:57.655377


# Create: enrich_memory

In [56]:
ContextEnricher.model.model_name = "us.meta.llama3-2-11b-instruct-v1:0"

In [57]:
%%time
from tqdm import tqdm
summaries = []
errors = []
for enum, context in enumerate(tqdm(new_contexts.contexts)):
    try:
        response = ContextEnricher.run(InputContext(context=context.context))
        summaries.append(
            Context(id=context.id, context=response.summary, metadata=context.metadata.copy())
        )
    except Exception as e:
        errors.append((enum, e, context))
len(summaries), len(errors)

100%|██████████| 63/63 [01:03<00:00,  1.02s/it]

CPU times: user 2.46 s, sys: 36.8 ms, total: 2.49 s
Wall time: 1min 3s


(63, 0)

In [58]:
enrich_memory.add_contexts(contexts=summaries)

In [59]:
enrich_memory.read_all().head(3)

,id,context,metadata,type,embedding,created_at,updated_at
0,c0563185-3f31-4070-b5fc-02a640ebbf9c,Researchers from Stanford University propose u...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.022033691, 0.005504608, -0.036621094, -0.0...",2025-04-28 00:03:29.039646,2025-04-28 00:03:29.039646
1,f7bb69b1-eacb-4727-bbb4-88eb928a812c,"Researchers propose STORM, a writing system to...","{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[-0.01360321, 0.021011353, -0.009269714, -0.01...",2025-04-28 00:03:30.059033,2025-04-28 00:03:30.059033
2,13092ee5-9276-4811-bab9-47cab9aed21d,The paper explores the challenges of using lar...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.01739502, 0.0095825195, -0.017318726, 0.00...",2025-04-28 00:03:31.257399,2025-04-28 00:03:31.257399


# Create: longterm_memory

In [63]:
combo_contexts = []
for n, s in zip(new_contexts.contexts, summaries):
    if n.id == s.id:
        combo_contexts.append(
            Context(id=n.id, context=f"{s.context}\n\n{n.context}", metadata=n.metadata.copy())
        )


In [64]:
len(new_contexts.contexts), len(summaries), len(combo_contexts)

(63, 63, 63)

In [65]:
longterm_memory.add_contexts(contexts=combo_contexts)

In [67]:
longterm_memory.read_all().head(3)

,id,context,metadata,type,embedding,created_at,updated_at
0,c0563185-3f31-4070-b5fc-02a640ebbf9c,Researchers from Stanford University propose u...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.04071045, 0.0022087097, -0.026824951, -0.0...",2025-04-28 00:04:42.348246,2025-04-28 00:04:42.348246
1,f7bb69b1-eacb-4727-bbb4-88eb928a812c,"Researchers propose STORM, a writing system to...","{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[-0.0007429123, -0.014602661, -0.023757935, -0...",2025-04-28 00:04:42.348280,2025-04-28 00:04:42.348280
2,13092ee5-9276-4811-bab9-47cab9aed21d,The paper explores the challenges of using lar...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.035980225, -0.0020961761, -0.021728516, -0...",2025-04-28 00:04:42.348293,2025-04-28 00:04:42.348293


# Create: jargon_memory

In [69]:
%%time
from tqdm import tqdm
jargons_contexts = []
errors = []
for enum, context in enumerate(tqdm(new_contexts.contexts[:])):
    try:
        response = JargonExtractor.run(InputContext(context=context.context))
        metadata = context.metadata.copy()
        for j in response.jargons:
            if j.evidence.lower() != "none":
                jr = JargonRecord(jargon=j.jargon, evidence=j.evidence, explanation=j.explanation, metadata=metadata.copy())
                jargons_contexts.append(jr)
    except Exception as e:
        print(enum, "="*20)
        errors.append(
            (enum, str(e), context)
        )

  0%|          | 0/63 [00:00<?, ?it/s]/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/ec2-user/SageMaker/broai-researcher-assistant/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the evidence of the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
100%|██████████| 63/63 [01:43<00:00,  1.65s/it]

CPU times: user 2.49 s, sys: 65 ms, total: 2.56 s
Wall time: 1min 43s


In [70]:
len(jargons_contexts), len(errors)

(134, 0)

In [71]:
jargons_contexts[0]

JargonRecord(id='1fa92eef-71a7-4987-b75c-04dd9f3b87a0', jargon='arXiv', evidence='# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024', explanation='arXiv is an online archive of electronic preprints', metadata={'section': '# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024', 'source': '.docs/test1/storm.md', 'type': 'document', 'sequence': 0}, created_at='2025-04-28 00:07:06.706352')

In [73]:
jargon_memory.add_jargons(jargons=jargons_contexts)

In [74]:
jargon_memory.filter_metadata(section="1 Introduction")

,id,jargon,evidence,explanation,metadata,created_at,updated_at
0,171df614-3e13-4c8a-b905-8fb44c00d084,LLMs,Large language models (LLMs) have demonstrated...,LLMs are large language models,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 00:07:10.002449,2025-04-28 00:07:10.002449
1,156c0940-6ddc-49ee-ace0-176c70466109,RAG,current strategies often involve retrieval-aug...,RAG is short for retrieval-augmented generation,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 00:07:10.002856,2025-04-28 00:07:10.002856
2,fd4d225c-6b35-4fb5-a9d8-98fa9bc2da3c,STORM,we propose the STORM paradigm for the Synthesi...,STORM is a paradigm for the Synthesis of Topic...,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 00:07:10.002880,2025-04-28 00:07:10.002880


In [84]:
jargon_memory.read_all()['jargon'].value_counts().head(20)

jargon
STORM                  22
LLM                    13
LLMs                    7
RAG                     6
API                     4
ACL                     4
AI                      3
NLP                     3
R                       3
oRAG                    3
GPT-3.5                 3
dspy                    2
Foo Fighters            2
Likert scale            2
DSPy                    2
Mistral 7B-Instruct     2
Sentence-BERT           2
EMNLP                   2
Siamese BERT            2
arXiv                   1
Name: count, dtype: int64

In [85]:
jargon_memory.fulltext_search(search_query="What does FreshWiki stand for?", context=False).to_dict(orient="records")

[{'id': 'e83946bd-57ef-4531-a932-98f1c246831e',
  'jargon': 'FreshWiki',
  'evidence': 'we randomly select 100 samples from the FreshWiki dataset',
  'explanation': 'FreshWiki is a dataset',
  'metadata': '{"section":"4 Experiments","source":".docs/test1/storm.md","type":"document","sequence":11}',
  'created_at': Timestamp('2025-04-28 00:07:25.157665'),
  'updated_at': Timestamp('2025-04-28 00:07:25.157665'),
  'score': 2.063894520567467}]

In [86]:
jargon_memory.fulltext_search(search_query="What does FreshWiki stand for?", context=True)

[JargonRecord(id='e83946bd-57ef-4531-a932-98f1c246831e', jargon='FreshWiki', evidence='we randomly select 100 samples from the FreshWiki dataset', explanation='FreshWiki is a dataset', metadata={'section': '4 Experiments', 'source': '.docs/test1/storm.md', 'type': 'document', 'sequence': 11}, created_at=Timestamp('2025-04-28 00:07:25.157665'))]

In [89]:
jargon_memory.fulltext_search(search_query="oRAG", context=False)

,id,jargon,evidence,explanation,metadata,created_at,updated_at,score
0,1ca97f48-82df-4a2c-939a-2b00ad876e70,oRAG,We conduct paired t-test and report the p-valu...,oRAG is a baseline method for generating articles,"{""section"":""6 Human Evaluation"",""source"":"".doc...",2025-04-28 00:07:43.088594,2025-04-28 00:07:43.088594,1.675271
1,2f157037-907c-4077-8cac-2daa5631e627,oRAG,*oRAG* significantly outperforms *RAG*,oRAG is a variant of RAG that uses outlines fo...,"{""section"":""5 Results and Analysis"",""source"":""...",2025-04-28 00:07:37.767475,2025-04-28 00:07:37.767475,1.675271
2,76721b7f-cb18-470b-8656-7bec5ce65ea4,oRAG,"- 3. *Outline-driven RAG (oRAG)*, which is ide...",oRAG is short for Outline-driven RAG,"{""section"":""4.3 Baselines"",""source"":"".docs/tes...",2025-04-28 00:07:29.006638,2025-04-28 00:07:29.006638,1.675271


In [91]:
jargon_memory.fulltext_search(search_query="STORM", context=False)["explanation"].tolist()

['STORM is a system that automates the pre-writing stage',
 'STORM is a paradigm for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking',
 'STORM is a system that simulates conversations',
 'STORM is a system or method for discovering perspectives and controlling question asking process',
 'STORM is a writing system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking']

In [92]:
jargon_memory.fulltext_search(search_query="oRAG", context=False)["explanation"].tolist()

['oRAG is a baseline method for generating articles',
 'oRAG is a variant of RAG that uses outlines for structuring full-length article generation',
 'oRAG is short for Outline-driven RAG']

In [93]:
jargon_memory.fulltext_search(search_query="LLM", context=False).to_dict(orient="records")

[{'id': '554a8780-fd0e-497d-864d-b64e6683a861',
  'jargon': 'LLM',
  'evidence': "The final outline is curated based on the LLM's intrinsic knowledge",
  'explanation': 'LLM is a large language model',
  'metadata': '{"section":"3 Method","source":".docs/test1/storm.md","type":"document","sequence":6}',
  'created_at': Timestamp('2025-04-28 00:07:16.359728'),
  'updated_at': Timestamp('2025-04-28 00:07:16.359728'),
  'score': 0.864510201497564},
 {'id': '171df614-3e13-4c8a-b905-8fb44c00d084',
  'jargon': 'LLMs',
  'evidence': 'Large language models (LLMs) have demonstrated impressive writing capabilities',
  'explanation': 'LLMs are large language models',
  'metadata': '{"section":"1 Introduction","source":".docs/test1/storm.md","type":"document","sequence":2}',
  'created_at': Timestamp('2025-04-28 00:07:10.002449'),
  'updated_at': Timestamp('2025-04-28 00:07:10.002449'),
  'score': 0.864510201497564},
 {'id': 'df4b2f6e-bf62-40bd-8929-1d8d45d4e1cc',
  'jargon': 'LLM',
  'evidence': 

In [94]:
jargon_memory.fulltext_search(search_query="STORM", context=False)

,id,jargon,evidence,explanation,metadata,created_at,updated_at,score
0,d49e02d7-670c-41fb-b437-f5f4defb0d99,STORM,We present STORM to automate the pre-writing s...,STORM is a system that automates the pre-writi...,"{""section"":""3 Method"",""source"":"".docs/test1/st...",2025-04-28 00:07:16.359357,2025-04-28 00:07:16.359357,0.821813
1,fd4d225c-6b35-4fb5-a9d8-98fa9bc2da3c,STORM,we propose the STORM paradigm for the Synthesi...,STORM is a paradigm for the Synthesis of Topic...,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-28 00:07:10.002880,2025-04-28 00:07:10.002880,0.821813
2,2ee2bf69-c67e-461a-8a47-0755848a96f1,STORM,STORM simulates a conversation between a Wikip...,STORM is a system that simulates conversations,"{""section"":""3.2 Simulating Conversations"",""sou...",2025-04-28 00:07:19.963609,2025-04-28 00:07:19.963609,0.821813
3,e5e8df9a-2cad-416f-9173-6c96ef178280,STORM,STORM discovers different perspectives by surv...,STORM is a system or method for discovering pe...,"{""section"":""3.1 Perspective-Guided Question As...",2025-04-28 00:07:18.240704,2025-04-28 00:07:18.240704,0.821813
4,9480f785-98f9-4a4b-bb72-fb1f20da28d0,STORM,"We propose STORM, a writing system for the Syn...",STORM is a writing system for the Synthesis of...,"{""section"":""Abstract"",""source"":"".docs/test1/st...",2025-04-28 00:07:08.075800,2025-04-28 00:07:08.075800,0.821813
